In [1]:
import pandas as pd
from indralib.indra_time import IndraTime
import re

In [2]:
# https://en.wikipedia.org/wiki/List_of_timelines

In [3]:
def remove_footnotes(text, numeric_only=True, single_letter_alpha=True):
    if numeric_only is False:
        text = re.sub(r"\[.*?\]", "", text)
    else:
        text = re.sub(r"\[\d+\]", "", text)
        if single_letter_alpha is True:
            text = re.sub(r"\[\w\]", "", text)
    return text

In [4]:
def extract_date_remarks(date):
    remarks = ""
    approxies = ["ca.", "c.", "circa", "around", "approximately", "~", ">", "<", "≈", "≥", "≤", "?"]
    for ap in approxies:
        if ap in date:
            date = date.replace(ap, "").strip()
            if date != "":
                remarks = f"{ap}"
    if ' or ' in date:
        idx = date.find(' or ')
        remarks = date[idx+1:] 
        date = date[:idx]
    if date.endswith('s'):
        remarks = date
        date = date[:-1]
    if "±" in date:
        dates = date.split("±")
        date = dates[0].strip()
        append = dates[1].strip().split(" ")
        if len(append) > 1:
            date = date + " " + append[1]
            remarks = "±" + append[0]
        else:
            remark = "±" + dates[1].strip()
    return date, remarks


In [5]:
def pre_clean(date):
    if isinstance(date, str):
        date = remove_footnotes(date)
        date = date.replace("–", " - ").replace("–", "-").replace(" to ", " - ").replace(",", "").replace("\xa0", " ").replace("  ", " ").replace("AD", "").replace("BCE", "BC").strip()
    return date

In [23]:
def decenturize(date):
    bc = False
    cent = False
    remarks = ""
    org_date = date
    postf = ['st', 'nd', 'rd', 'th']
    if ' century BC' in date:
        bc = True
        fpf = False
        for pf in postf:
            if pf in date:
                date = date.replace(pf, '')
                fpf = True
                break
        if fpf is True:
            date = date.replace(' century BC', '')
            cent = True
    elif ' century' in date:
        if 'c. ' in date:
            date = date.replace("c. ", "")
        fpf = False
        for pf in postf:
            if pf in date:
                date = date.replace(pf, '')
                fpf = True
                break
        if fpf is True:
            date = date.replace(' century', '')
            cent = True
    if cent is False:
        return org_date, remarks
    century = date.strip()
    try:
        int_cent = int(century)
    except:
        return org_date, remarks
    if bc is True:
        if int_cent == 1:
            date = f"100 BC - 1 BC"
        else:
            date = f"{int_cent}00 BC - {int_cent-1}00 BC"
    else:
        if int_cent == 1:
            date = "1 - 100"
        else:
            date = f"{int_cent-1}00 - {int_cent}00"
    return date, org_date

In [7]:
def date_clean(date, default_scale = None):
    remarks = ""
    if isinstance(date, str):
        date = pre_clean(date)
        date_sub = date.split("-")
        if len(date_sub)==3 or (len(date_sub)==2 and " - " not in date):
            return date, remarks
        if default_scale is not None:
            date = date.split("-")
            if len(date) == 1:
                date = date[0].strip()+f" {default_scale}"
            else:
                date = f"{date[0].strip()} {default_scale} - {date[1].strip()} {default_scale}"
        else:
            date, remarks = decenturize(date)
            dates = date.split(" - ")
            new_dates = []
            for di in dates:
                dj, rem = extract_date_remarks(di)
                new_dates.append(dj)
                if len(rem)>0:
                    if len(remarks) == 0:
                        remarks = rem
                    else:
                        remarks += f", {rem}"
            date = " - ".join(new_dates)
            dates = date.split(" - ")
            if len(dates) == 2:
                sub_dates0 = dates[0].split(" ")
                sub_dates1 = dates[1].split(" ")
                if len(sub_dates0)==1 and len(sub_dates1)==2:
                    date = f"{sub_dates0[0]} {sub_dates1[1]} - {dates[1]}"
    else:
        if default_scale is not None:
            date = f"{date} {default_scale}"
        else:
            date = str(int(date))
    if "/" in date:
        date_parts = date.split("/")
        date = date_parts[0].strip()
        alt_date_stub = date_parts[1].strip()
        alt_date = date[:len(date)-len(alt_date_stub)] + alt_date_stub
        remarks = f"Alt.: {alt_date}"       
    jd_dates = IndraTime.string_time_to_julian(date)
    
    if len(jd_dates) > 1 and jd_dates[1] is not None:
        date_start = IndraTime.julian_to_string_time(jd_dates[0])
        date_end = IndraTime.julian_to_string_time(jd_dates[1])
        date = f"{date_start} - {date_end}"
    else:
        date = IndraTime.julian_to_string_time(jd_dates[0])

    return date, remarks

In [8]:
def date_merge(year, rest):
    day = None
    month = None
    year = pre_clean(year)
    rest = rest.strip()
    year = year.strip()
    if rest is not None and len(rest) > 0:
        rparts = rest.split(" ")
        if len(rparts) == 1:
            month = rparts[0].strip().lower()
        else:
            try:
                month = rparts[1].strip().lower()
                day = int(rparts[0].strip())
            except ValueError:
                month = None
                day = None
            if month is None and day is None:
                try:
                    month = rparts[0].strip().lower()
                    day = int(rparts[1].strip())
                except ValueError:
                    month = None
                    day = None
    else:
        month = None
        day = None
    if month is not None:
        val_months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
        try:
            month_id = val_months.index(month) + 1
        except ValueError:
            month_id = None
            return f"{year}"
        year_parts = str(year).replace("\xa0"," ").split(" ", 1)
        if len(year_parts) > 1:
            year = year_parts[0]
            appendix = year_parts[1]
        else:
            appendix = ""
        if day is not None:
            date = f"{year}-{month_id:02d}-{day:02d}"
            if appendix != "":
                date = date + f" {appendix}"
        else:
            date = f"{year}-{month_id:02d}"
            if appendix != "":
                date = date + f" {appendix}"
        return date
    else:
        return f"{year}"

In [112]:
url = "https://en.wikipedia.org/wiki/Timeline_of_Iranian_history"
tables = pd.read_html(url)

In [17]:
tables[0]

,Year,Date,Event
0,3200 BC,NaN,Elam civilization in the far west and southwes...


In [18]:
tables[32]

,Year,Date,Event
0,2001,8 June,"Iranian presidential election, 2001: President..."
1,2003,December,"40,000 people are killed in an earthquake in s..."
2,2005,24 June,"Iranian presidential election, 2005: Ahmadinej..."
3,2009,12 June,"Iranian presidential election, 2009: Ahmadinej..."
4,2009,13 June,2009–10 Iranian election protests: Protests in...
5,2013,3 August,Hassan Rouhani replaces Ahmadinejad as President.
6,2014,NaN,"My Stealthy Freedom, an online movement in whi..."
7,2015,14 July,Signing of Iran nuclear deal between Iran and ...
8,2018,8 May,United States withdraws from the Iran nuclear ...
9,2017–19,NaN,Iranian woman protest against compulsory hijab...


In [69]:
print("| Date | Event in Persian (Iranian) history |")
print("| ---- | ---- |")
for index in range(0,33):
    for i, row in enumerate(tables[index].iterrows()):
        year = str(row[1].iloc[0])
        if pd.isna(year) or year == "nan":
            continue
        year = remove_footnotes(year)
        rest = row[1].iloc[1]
        if pd.isna(rest):
            rest = ""
        rest = remove_footnotes(rest)
        date = date_merge(year, rest)
        date, remarks = date_clean(date)
        event = row[1].iloc[2]
        event = remove_footnotes(event)
        if pd.isna(event):
            continue
        if remarks != "":
            event = f"Date: {remarks}, {event}"
        print(f"| {date} | {event} |")

| Date | Event in Persian (Iranian) history |
| ---- | ---- |


ValueError: Invalid date format: prehistoryprehistory of anatolia palaeolithic anatolia  500000

In [20]:
url = "https://en.wikipedia.org/wiki/Egyptian_pyramids"
tables = pd.read_html(url)

In [21]:
tables[12]

,Pyramid (Pharaoh),Reign,Field,Height
0,Pyramid of Djoser (Djoser),c. 2670 BCE,Saqqara,62 meters (203 feet)
1,Red Pyramid (Sneferu),c. 2612–2589 BCE,Dahshur,104 meters (341 feet)
2,Meidum Pyramid (Sneferu),c. 2612–2589 BCE,Meidum,65 meters (213 feet) (ruined) Would have been ...
3,Great Pyramid of Giza (Khufu),c. 2589–2566 BCE,Giza,146.7 meters (481 feet) or 280 Egyptian Royal ...
4,Pyramid of Djedefre (Djedefre),c. 2566–2558 BCE,Abu Rawash,60 meters (197 feet)
5,Pyramid of Khafre (Khafre),c. 2558–2532 BCE,Giza,136.4 meters (448 feet) Originally: 143.5 m (4...
6,Pyramid of Menkaure (Menkaure),c. 2532–2504 BCE,Giza,65 meters (213 feet) or 125 Egyptian Royal cubits
7,Pyramid of Userkaf (Userkaf),c. 2494–2487 BCE,Saqqara,48 meters (161 feet)
8,Pyramid of Sahure (Sahure),c. 2487–2477 BCE,Abusir,47 meters (155 feet)
9,Pyramid of Neferirkare (Neferirkare Kakai),c. 2477–2467 BCE,Abusir,72.8 meters (239 feet)


In [25]:
print("| Date | Pyramid | Pharaoh | Field | Height |")
print("| ---- | ---- | ---- | ---- | ---- |")
for i, row in enumerate(tables[12].iterrows()):
    date = row[1].iloc[1]
    if pd.isna(date) or date == "nan":
        continue
    date = remove_footnotes(str(date))
    date, remarks = date_clean(date)
    pyramid = row[1].iloc[0]
    pyramid = remove_footnotes(pyramid)
    pyr_phar = pyramid.split("(")
    pyramid = pyr_phar[0].strip()
    if len(pyr_phar) > 1:
        pharao = pyr_phar[1].split(")")[0]
    else:
        pharao = ""
    if pd.isna(pyramid):
        continue
    field = row[1].iloc[2]
    field = remove_footnotes(field)
    height = row[1].iloc[3]
    height = remove_footnotes(height)
    if remarks != "":
        pyramid = f"Date: {remarks}, {pyramid}"
    print(f"| {date} | {pyramid} | {pharao} | {field} | {height} |")

| Date | Pyramid | Pharaoh | Field | Height |
| ---- | ---- | ---- | ---- | ---- |
| 2670 BC | Date: c., Pyramid of Djoser | Djoser | Saqqara | 62 meters (203 feet) |
| 2612 BC - 2589 BC | Date: c., , Red Pyramid | Sneferu | Dahshur | 104 meters (341 feet) |
| 2612 BC - 2589 BC | Date: c., , Meidum Pyramid | Sneferu | Meidum | 65 meters (213 feet) (ruined) Would have been 91.65 meters (301 feet)[citation needed] or 175 Egyptian Royal cubits. |
| 2589 BC - 2566 BC | Date: c., , Great Pyramid of Giza | Khufu | Giza | 146.7 meters (481 feet) or 280 Egyptian Royal cubits |
| 2566 BC - 2558 BC | Date: c., , Pyramid of Djedefre | Djedefre | Abu Rawash | 60 meters (197 feet) |
| 2558 BC - 2532 BC | Date: c., , Pyramid of Khafre | Khafre | Giza | 136.4 meters (448 feet) Originally: 143.5 m (471 ft) or 274 Egyptian Royal cubits |
| 2532 BC - 2504 BC | Date: c., , Pyramid of Menkaure | Menkaure | Giza | 65 meters (213 feet) or 125 Egyptian Royal cubits |
| 2494 BC - 2487 BC | Date: c., , Pyramid

In [27]:
url= "https://en.wikipedia.org/wiki/Dynasties_of_ancient_Egypt"
tables = pd.read_html(url)

In [37]:
tables[12]

Dynasty  \
                                              Dynasty   
                                Early Dynastic Period   
0                                           Dynasty I   
1                                          Dynasty II   
2                                         Old Kingdom   
3                                         Dynasty III   
4                                          Dynasty IV   
5                                           Dynasty V   
6                                          Dynasty VI   
7                           First Intermediate Period   
8                                      Dynasty VII[d]   
9                                        Dynasty VIII   
10                                         Dynasty IX   
11                                          Dynasty X   
12                                     Middle Kingdom   
13                                      Dynasty XI[f]   
14                                        Dynasty XII   
15                                    Dynasty XIII[g]   
16                         Second Intermediate Period   
17                                        Dynasty XIV   
18                                Dynasty XV (Hyksos)   
19                                  Abydos dynasty[i]   
20                                        Dynasty XVI   
21                                       Dynasty XVII   
22                                        New Kingdom   
23                                      Dynasty XVIII   
24                                        Dynasty XIX   
25                                         Dynasty XX   
26                          Third Intermediate Period   
27                                        Dynasty XXI   
28                            Dynasty XXII (Meshwesh)   
29                           Dynasty XXIII (Meshwesh)   
30                                       Dynasty XXIV   
31                               Dynasty XXV (Nubian)   
32                                        Late Period   
33                                       Dynasty XXVI   
34                         Dynasty XXVII[j] (Persian)   
35                                     Dynasty XXVIII   
36                                       Dynasty XXIX   
37                                        Dynasty XXX   
38                          Dynasty XXXI[m] (Persian)   
39                                 Greco–Roman Period   
40                             Argead dynasty (Greek)   
41                          Ptolemaic dynasty (Greek)   
42  Egypt was incorporated into the Roman Republic...   

                                                 Seat  \
                                                 Seat   
                                Early Dynastic Period   
0                                              Thinis   
1                                              Thinis   
2                                         Old Kingdom   
3                                             Memphis   
4                                             Memphis   
5                                             Memphis   
6                                             Memphis   
7                           First Intermediate Period   
8                                     Memphis[1]: 396   
9                                     Memphis[1]: 396   
10                                Heracleopolis Magna   
11                                Heracleopolis Magna   
12                                     Middle Kingdom   
13                                             Thebes   
14                                            Itjtawy   
15                                            Itjtawy   
16                         Second Intermediate Period   
17                                             Avaris   
18                                             Avaris   
19                                             Abydos   
20                                   Thebes or Avaris   
21                                             Thebes  

In [41]:
print("| Date | Egyptian dynasty | Seat | First to rule | Last to rule | Duration (years) |")
print("| ---- | ---- | ---- | ---- | ---- | ---- |")  
for i, row in enumerate(tables[12].iterrows()):
    start = row[1].iloc[2]
    end = row[1].iloc[3]
    if start == end:
        continue  # skip the header
    date = f"{start} - {end}"
    date = remove_footnotes(str(date))
    date, remarks = date_clean(date)
    dynasty = remove_footnotes(row[1].iloc[0])
    seat = remove_footnotes(row[1].iloc[1])
    duration = row[1].iloc[4].split(" ")[0]
    first = remove_footnotes(row[1].iloc[5])
    last = remove_footnotes(row[1].iloc[6])
    if remarks != "":
        dynasty = f"Date: {remarks}, {dynasty}"
    print(f"| {date} | {dynasty} | {seat} | {first} | {last} | {duration} |")

| Date | Egyptian dynasty | Seat | First to rule | Last to rule | Duration (years) |
| ---- | ---- | ---- | ---- | ---- | ---- |
| 3150 BC - 2900 BC | Dynasty I | Thinis | Narmer | Qa'a | 250 |
| 2880 BC - 2686 BC | Dynasty II | Thinis | Hotepsekhemwy | Khasekhemwy | 204 |
| 2687 BC - 2613 BC | Dynasty III | Memphis | Djoser | Huni | 73 |
| 2613 BC - 2494 BC | Dynasty IV | Memphis | Sneferu | Shepseskaf or Thamphthis | 112 |
| 2494 BC - 2345 BC | Dynasty V | Memphis | Userkaf | Unas | 149 |
| 2345 BC - 2181 BC | Dynasty VI | Memphis | Teti | Merenre Nemtyemsaf II or Netjerkare Siptah or Nitocris | 164 |
| 2181 BC - 2160 BC | Dynasty VIII | Memphis: 396 | Netjerkare Siptah or Menkare | Neferirkare II | 21 |
| 2160 BC - 2130 BC | Dynasty IX | Heracleopolis Magna | Meryibre Khety | Unknown | 30 |
| 2130 BC - 2040 BC | Dynasty X | Heracleopolis Magna | Meryhathor | Unknown | 90 |
| 2130 BC - 1991 BC | Dynasty XI | Thebes | Intef | Mentuhotep IV | 139 |
| 1991 BC - 1802 BC | Dynasty XII | I

In [9]:
url = "https://en.wikipedia.org/wiki/Bond_event"
tables = pd.read_html(url)

In [10]:
len(tables)

15

In [11]:
tables[0]

,No,Time (BP),"Time (AD, BC)",Gap from previous event,Notes
0,0,≈ −0.5 ka,≈ 1500 AD,900 years,See Little Ice Age[12]
1,1,≈ −1.4 ka,≈ 600 AD,1400 years,See Migration Period[12] and Late Antique Litt...
2,2,≈ −2.8 ka,≈ 800 BC,1400 years,See Iron Age Cold Epoch
3,3,≈ −4.2 ka,≈ 2200 BC,1700 years,See 4.2-kiloyear event; collapse of the Akkadi...
4,4,≈ −5.9 ka,≈ 3900 BC,2300 years,"Sahara desert reforms by 3500–3000 BC, ending ..."
5,5,≈ −8.2 ka,≈ 6200 BC,1200 years,See 8.2-kiloyear event
6,6,≈ −9.4 ka,≈ 7400 BC,1100 years,"Erdalen event of glacier activity in Norway,[1..."
7,7,≈ −10.3 ka,≈ 8300 BC,800 years,NaN
8,8,≈ −11.1 ka,≈ 9100 BC,—,Transition from the Younger Dryas to the Borea...


In [14]:
print("| Date | Bond event name | Id |")
print("| --- | --- | --- |")
for i, row in enumerate(tables[0].iterrows()):
    no = row[1].iloc[0]
    date = row[1].iloc[2]
    name = row[1].iloc[4]
    if pd.isna(date) or pd.isna(name):
        name = ""
    else:
       name = remove_footnotes(name)
       name = name.replace("See ", "")
    
    date, remark = date_clean(date)
    if remark != "":
        name = f"Date: {remark}, {name}"
    print(f"| {date} | {name} | {no} |")

| Date | Bond event name | Id |
| --- | --- | --- |
| 1500 | Date: ≈, Little Ice Age | 0 |
| 600 | Date: ≈, Migration Period and Late Antique Little Ice Age | 1 |
| 800 BC | Date: ≈, Iron Age Cold Epoch | 2 |
| 2200 BC | Date: ≈, 4.2-kiloyear event; collapse of the Akkadian Empire and the end of the Egyptian Old Kingdom. | 3 |
| 3900 BC | Date: ≈, Sahara desert reforms by 3500–3000 BC, ending Neolithic Subpluvial. Piora Oscillation. Early Bronze Age begins ~3300 BC. | 4 |
| 6200 BC | Date: ≈, 8.2-kiloyear event | 5 |
| 7400 BC | Date: ≈, Erdalen event of glacier activity in Norway, as well as a cold event in China. | 6 |
| 8300 BC | Date: ≈,  | 7 |
| 9100 BC | Date: ≈, Transition from the Younger Dryas to the Boreal. | 8 |


In [24]:
url = "https://en.wikipedia.org/wiki/Timeline_of_Anatolian_history"
tables = pd.read_html(url)

In [32]:
tables[0]

,Part of a series on the,Unnamed: 1_level_0
,History of Turkey,Unnamed: 1_level_1
0,NaN,NaN
1,PrehistoryPrehistory of Anatolia Palaeolithic...,NaN
2,Palaeolithic Anatolia,"c. 500,000– 10,000 BC"
3,Mesolithic Anatolia,"c. 11,000– 9,000 BC"
4,Neolithic Anatolia,"c. 8,000– 5,500 BC"
...,...,...
83,Multi-party period,(1945–present)
84,By topic Agriculture Anatolian peoples Christi...,NaN
85,Timeline,NaN


In [70]:
print("| Date | Event in history of Anatolia (Turkey) |")
print("| ---- | ---- |")
hist = []
for index in range(0,7):
    for i, row in enumerate(tables[index].iterrows()):
        year = row[1].iloc[1]
        if isinstance(year, str):
            year = year.replace('–present','')
            year = year.replace('-27 BC–', ' - ')
            if year.startswith('(') and year.endswith(')'):
                year = year[1:-1]
            year = year.split('(')[0]
            year = year.split(')')[0]
            year = year.split(';')[0]
        if pd.isna(year) or year == "nan":
            continue
        year, remarks = date_clean(year)
        event = row[1].iloc[0]
        if pd.isna(event):
            continue
        if remarks != "":
            event = f"Date: {remarks}{event}"
        jd_dates = IndraTime.string_time_to_julian(year)
        hist.append((jd_dates, event))
        # print(f"| {year} | {event} |")
for index in range(8,31):
    for i, row in enumerate(tables[index].iterrows()):
        year = row[1].iloc[0]
        if pd.isna(year) or year == "nan":
            continue
        if isinstance(year, str) is False:
            year = str(int(year))
        year = remove_footnotes(year)
        rest = row[1].iloc[1]
        if pd.isna(rest):
            rest = ""
        rest = remove_footnotes(rest)
        date = date_merge(year, rest)
        date, remarks = date_clean(date)
        event = row[1].iloc[2]
        if pd.isna(event):
            continue
        event = remove_footnotes(event)
        if pd.isna(event):
            continue
        if remarks != "":
            event = f"Date: {remarks}{event}"
        jd_dates = IndraTime.string_time_to_julian(date)
        hist.append((jd_dates, event))
        # print(f"| {date} | {event} |")

entries = sorted(hist, key=lambda x: x[0][0])
old_dates=None
old_event = None
for entry in entries:
    # print(entry[0])
    date_1 = IndraTime.julian_to_string_time(entry[0][0])
    if len(entry[0])>1 and entry[0][1]!=entry[0][0] and entry[0][1] != None:
        date_2 = IndraTime.julian_to_string_time(entry[0][1])
        dates = f"{date_1} - {date_2}"
    else:
        dates = f"{date_1}"
    event = entry[1]
    if dates==old_dates and event==old_event:
        # print("DUPE!")
        pass
    else:
        print(f"| {dates} | {event} |")
    old_dates = dates
    old_event = event
    

| Date | Event in history of Anatolia (Turkey) |
| ---- | ---- |
| 501.95 kya BP - 10000 BC | Date: c., Palaeolithic Anatolia |
| 11000 BC - 9000 BC | Date: c., Mesolithic Anatolia |
| 8000 BC - 5500 BC | Date: c., Neolithic Anatolia |
| 3000 BC - 700 BC | Troy |
| 2500 BC - 2000 BC | Hattians |
| 2400 BC - 2150 BC | Akkadian Empire |
| 2300 BC - 1400 BC | Luwians |
| 1950 BC - 1750 BC | Assyria |
| 1780 BC - 1680 BC | Kussara |
| 1700 BC - 1300 BC | Achaeans (Homer) |
| 1680 BC - 1220 BC | Hittites |
| 1650 BC - 1450 BC | Kizzuwatna |
| 1600 BC | Hittite Empire established in the region of Anatolia with capital in Hattusa near present-day Boğazkale, Turkey. |
| 1500 BC - 1320 BC | Arzawa |
| 1500 BC - 1300 BC | Mitanni |
| 1500 BC - 1290 BC | Hayasa-Azzi |
| 1500 BC - 1320 BC | Arzawa |
| 1500 BC - 1300 BC | Mitanni |
| 1500 BC - 1290 BC | Hayasa-Azzi |
| 1450 BC - 350 BC | Lycia |
| 1346 BC | Hittite empire reaches its height under the rule of Šuppiluliuma I. |
| 1300 BC - 1250 BC | 

In [72]:
url = "https://en.wikipedia.org/wiki/Timeline_of_Romanian_history"
tables = pd.read_html(url)

In [81]:
tables[28]

,Year,Date,Event
0,2004,NaN,Romania joins the NATO. Traian Băsescu becomes...
1,2007,NaN,"On January 1, Romania joins the European Union..."
2,2008,NaN,In February the Government overrule court deci...
3,2009,NaN,"Badly affected by the Late-2000s recession, th..."
4,2013,NaN,Large protests against Prime Minister Victor P...
5,2014,December 21,Klaus Iohannis becomes the fifth President of ...


In [92]:
print("| Date | Event in history of Romania |")
print("| ---- | ---- |")
for index in range(3,29):
    for i, row in enumerate(tables[index].iterrows()):
        year = row[1].iloc[0]
        if pd.isna(year) or year == "nan":
            continue
        if isinstance(year, str) is False:
            year = str(int(year))
        year = remove_footnotes(year)
        if year == "Year":
            continue
        rest = row[1].iloc[1]
        if pd.isna(rest):
            rest = ""
        rest = remove_footnotes(rest)
        date = date_merge(year, rest)
        date, remarks = date_clean(date)
        event = row[1].iloc[2]
        if pd.isna(event):
            continue
        event = remove_footnotes(event)
        if pd.isna(event):
            continue
        if remarks != "":
            event = f"Date: {remarks}{event}"
        print(f"| {date} | {event} |")

| Date | Event in history of Romania |
| ---- | ---- |
| 440 BC | Herodotus in Histories is the first to mention Dacians as living on the territory of modern-day Romania. |
| 400 - 500 | Date: 5th centuryCharnabon, king of the Getae. |
| 400 - 500 | Date: 5th centuryThe Geto-Dacian Helmet of Peretu. |
| 339 | Rex Histrianorum, ruling over Histria, is mentioned by Trogus Pompeius și Justinus[citation needed]. |
| 300 - 400 | Date: 4th centuryThe Getae silver Helmet of Agighiol. |
| 300 - 400 | Date: 4th centuryThe Geto-Dacian Helmet of Coțofenești. |
| 300 - 400 | Date: 4th centuryThe Geto-Dacian Helmet of Iron Gates. |
| 300 - 400 | Date: 4th centuryThe Geto-Dacian Helmet of Făcău-Bulbucata. |
| 200 BC | King Rhemaxos protected Greek colonies in Dobruja. |
| 100 BC | King Dicomes ruled Dacia. |
| 100 BC | King Rholes ruled Dobruja.[citation needed] |
| 100 BC | King Dapyx ruled Dobruja.[citation needed] |
| 82 BC | King Burebista rules Dacia. (to 44 BC)[citation needed] |
| 44 BC | Dec

In [93]:
url = "https://en.wikipedia.org/wiki/Timeline_of_Spanish_history"
tables = pd.read_html(url)

In [99]:
tables[16]

,Year,Date,Event
0,2003,22 August,"Fernando Alonso wins the Hungarian Grand Prix,..."
1,2004,11 March,2004 Madrid train bombings: Madrid train bombi...
2,2004,14 March,Aznar's People's Party lost an election after ...
3,2005,NaN,PADICAT (archive) established.
4,2005,25 September,Fernando Alonso wins the Championship against ...
5,2006,30 December,2006 Madrid–Barajas Airport bombing: A bombing...
6,2008,NaN,Moroccan national Jamal Zougam was found guilt...
7,2008,NaN,Garzón was charged with criminal conduct in th...
8,2008,NaN,Spain won the UEFA European Championship Final...
9,2010,July,Spain won the FIFA World Cup.[13]


In [101]:
print("| Date | Event in history of Spain |")
print("| ---- | ---- |")
for index in range(1,17):
    for i, row in enumerate(tables[index].iterrows()):
        year = row[1].iloc[0]
        if pd.isna(year) or year == "nan":
            continue
        if isinstance(year, str) is False:
            year = str(int(year))
        year = remove_footnotes(year)
        if year == "Year":
            continue
        rest = row[1].iloc[1]
        if pd.isna(rest):
            rest = ""
        rest = remove_footnotes(rest)
        date = date_merge(year, rest)
        date, remarks = date_clean(date)
        event = row[1].iloc[2]
        if pd.isna(event):
            continue
        event = remove_footnotes(event)
        if pd.isna(event):
            continue
        if remarks != "":
            event = f"Date: {remarks}{event}"
        print(f"| {date} | {event} |")

| Date | Event in history of Spain |
| ---- | ---- |
| 507 | The Franks attacked allied with the Burgundians to the Visigothic Kingdom of Toulouse. Alaric II was killed at the Battle of Vouillé, and Toulouse was sacked. The Visigoths had lost most of their Gallic holdings and are retreated to Hispania and Septimania, helped by Ostrogoths. Kingdom of Toulouse ended and The Arian Kingdom of Hispania began. Barcelona was the new capital of the Visigoths. From this moment, little by little, Visigothic Hispania will be the first effective realisation of an independent Kingdom or State of wholly Hispanic territories and scope. |
| 552 | Byzantine Empire conquered part of the south of the Visigothic kingdom. |
| 567 | Toledo. Capital of the Visigothic kingdom by the end of the reign of Athanagild. |
| 568 | Liuvigild began his reign. He is among the greatest Visigothic kings of the Arian period because he consolidated Visigothic power in Spain. He is known for his Code of Leovigild, a law tha

In [113]:
url = "https://en.wikipedia.org/wiki/Timeline_of_Portuguese_history"
tables = pd.read_html(url)

In [114]:
tables[19]

,Year,Date,Event
0,2001,4 March,Hintze Ribeiro Bridge disaster: 59 people die ...
1,2002,1 January,Portugal adopts the euro as currency.
2,2004,12 June – 4 July,2004 European Football Championship is held in...
3,2005,31 December,"The 2006 Dakar Rally, the longest and, arguabl..."
4,2007,11 February,"In the second Portuguese abortion referendum, ..."
5,2010,17 May,The law that allows the same-sex marriage is a...
6,2010,NaN,"In 2010, the official infant mortality rate wa..."
7,2016,10 July,Portugal national football team wins the 2016 ...
8,2017,13 May,Salvador Sobral wins the 62nd edition of the E...
9,2018,8 – 12 May,Lisbon hosts the 63rd edition of the Eurovisio...


In [122]:
print("| Date | Event in history of Portugal |")
print("| ---- | ---- |")
for index in range(0,20):
    for i, row in enumerate(tables[index].iterrows()):
        year = row[1].iloc[0]
        if pd.isna(year) or year == "nan":
            continue
        if isinstance(year, str) is False:
            year = str(int(year))
        year = remove_footnotes(year)
        if year == "Year":
            continue
        rest = row[1].iloc[1]
        if pd.isna(rest):
            rest = ""
        rest = remove_footnotes(rest)
        date = date_merge(year, rest)
        date, remarks = date_clean(date)
        event = row[1].iloc[2]
        if pd.isna(event):
            continue
        event = remove_footnotes(event)
        if pd.isna(event):
            continue
        if remarks != "":
            event = f"Date: {remarks}, {event}"
        print(f"| {date} | {event} |")

| Date | Event in history of Portugal |
| ---- | ---- |
| 218 BC - 201 BC | The Second Punic War would bring the Roman Republic's influence to Iberia. The Roman conquest of the Iberian Peninsula begins. |
| 197 BC | The Iberian Peninsula is divided into Hispania Ulterior (which will include all of Portugal) and Hispania Citerior. |
| 155 BC | Resistance to the Roman conquest by the Lusitanians starts in the Lusitanian War. Punicus leads the Lusitanians resistance, while Manilius and Calpurnius Piso are the local praetors. |
| 155 BC | After their first victory, the Lusitanians ally themselves with the Vettones and lay siege to a Phoenician settlement, the Blastophoenicians, who are subjects of Rome. Punicus is killed during this siege and is succeeded by Caesarus. |
| 155 BC | Mummius is sent to Rome and fights Caesarus. |
| 153 BC | The Lusitanians on the other side of the Tagus were led by Caucenus and join the resistance, invading the Cunei, who were subject to Rome, and capturing C

In [123]:
url = "https://en.wikipedia.org/wiki/Timeline_of_Indian_history"
tables = pd.read_html(url)

In [128]:
tables[44]

,Year,Date,Event
0,2001,26 January,The 7.7 Mw Gujarat earthquake shakes Western I...
1,2001,July,Vajpayee meets Pakistani President Pervez Mush...
2,2001,July,Vajpayee's BJP party declines his offer to res...
3,2001,September,US lifts sanctions which it imposed against In...
4,2001,October,India and Pakistan fire at each other's milita...
...,...,...,...
73,2024,29 June,India defeats South Africa and wins the 2024 I...
74,2024,NaN,NaN
75,2024,NaN,NaN
76,2024,NaN,NaN


In [129]:
print("| Date | Event in history of India |")
print("| ---- | ---- |")
for index in range(0,45):
    for i, row in enumerate(tables[index].iterrows()):
        year = row[1].iloc[0]
        if pd.isna(year) or year == "nan":
            continue
        if isinstance(year, str) is False:
            year = str(int(year))
        year = remove_footnotes(year)
        if year == "Year":
            continue
        rest = row[1].iloc[1]
        if pd.isna(rest):
            rest = ""
        rest = remove_footnotes(rest)
        date = date_merge(year, rest)
        date, remarks = date_clean(date)
        event = row[1].iloc[2]
        if pd.isna(event):
            continue
        event = remove_footnotes(event)
        if pd.isna(event):
            continue
        if remarks != "":
            event = f"Date: {remarks}, {event}"
        print(f"| {date} | {event} |")

| Date | Event in history of India |
| ---- | ---- |
| 4001.95 kya BP - 101.95 kya BP | Tools crafted by proto-humans that have been dated back two million years have been discovered in the northwestern subcontinent. The earliest archaeological site in the subcontinent is the palaeolithic hominid site in the Soan River valley. Soanian sites are found in the Sivalik region across what are now India, Pakistan, and Nepal. Some of the Bhimbetka rock shelters were inhabited by Homo erectus more than 100,000 years ago. Evidence suggested that occupation of the Indian subcontinent by hominins was sporadic until circa 700,000 years ago, and was geographically widespread by around 250,000 years ago. Madrasian culture sites have been found in Attirampakkam (Attrambakkam=13° 13' 50", 79° 53' 20"), which is located near Chennai (formerly known as Madras), Tamil Nadu. Thereafter, tools related to this culture have been found at various other locations in this region. Bifacial handaxes and cleavers 

In [3]:
url = "https://en.wikipedia.org/wiki/Timeline_of_Roman_history"
tables = pd.read_html(url)

In [7]:
tables[21]

,Year,Date,Event
0,1453,29 May,Fall of Constantinople: Ottoman forces entered...


In [21]:
print("| Date | Event in history of the Roman Empire |")
print("| ---- | ---- |")
for index in range(0,22):
    for i, row in enumerate(tables[index].iterrows()):
        year = row[1].iloc[0]
        if pd.isna(year) or year == "nan":
            continue
        if isinstance(year, str) is False:
            year = str(int(year))
        year = remove_footnotes(year)
        if year == "Year":
            continue
        rest = row[1].iloc[1]
        if pd.isna(rest):
            rest = ""
        rest = remove_footnotes(rest)
        date = date_merge(year, rest)
        date, remarks = date_clean(date)
        event = row[1].iloc[2]
        if pd.isna(event):
            continue
        event = remove_footnotes(event)
        if pd.isna(event):
            continue
        if remarks != "":
            event = f"Date: {remarks}, {event}"
        print(f"| {date} | {event} |")

| Date | Event in history of the Roman Empire |
| ---- | ---- |
| 754 BC | Battle of Alba Longa. King of Alba Longa, Amulius, who had previously usurped power is defeated and killed along with his sons by his Brother Numitor, and great-nephew Romulus, who led a sizable warband. |
| 753-04-21 BC | Rome was founded. According to Roman legend, Romulus was the founder and first King of Rome, establishing the Roman Kingdom. |
| 752 BC | Romulus, first king of Rome, celebrates the first Roman triumph after his victory over the Caeninenses, following the Rape of the Sabine Women. He celebrates a further triumph later in the year over the Antemnates. |
| 752 BC | Rome's first colonies were established. |
| 715 BC | Numa Pompilius became the second King of Rome. |
| 673 BC | Tullus Hostilius became the third King of Rome. |
| 667 BC | Byzantium was founded by Megarian colonists. |
| 642 BC | Tullus Hostilius died. |
| 642 BC | The Curiate Assembly, one of the legislative assemblies of the Roman

In [22]:
url="https://en.wikipedia.org/wiki/Timeline_of_post-classical_history"
tables = pd.read_html(url)

In [35]:
tables[3]

,Year,Date,Event,Significance
0,c. 400,NaN,Highland Maya fall to the lowland city of Teot...,Begins the decline of Maya culture and languag...
1,405,NaN,St. Jerome finished the Vulgate.,The Christian Gospel is translated into Latin.[4]
2,410,August 24,"Rome is sacked by Alaric, King of the Visigoths",Decisive event in the decline of the Western R...
3,431,June 22 – July 31,Council of Ephesus,"Confirmed the original Nicene Creed,[6] and co..."
4,455,June 2,"Rome is sacked by Genseric, King of the Vandal...",Another decisive event in the Fall of Rome and...
5,476,September 4,Odoacer deposes the Roman Emperor Romulus Augu...,Considered by some historians to be the starti...
6,480,April 25,"Death of Julius Nepos, last Roman Emperor to b...",Considered by some historians to be the starti...
7,493,March 15,Theodoric the Great killed Odoacer at a banque...,The Ostrogothic Kingdom ruled Italy from 493 t...
8,c. 500,NaN,Tikal becomes the first great Maya city,Significant cultural exchange between the Maya...
9,c. 500,NaN,Battle of Mons Badonicus.,The West Saxon advance is halted by Britons in...


In [36]:
print("| Date | Post-classical history event |")
print("| ---- | ---- |")
for index in range(3,13):
    for i, row in enumerate(tables[index].iterrows()):
        year = row[1].iloc[0]
        if pd.isna(year) or year == "nan":
            continue
        if isinstance(year, str) is False:
            year = str(int(year))
        year = remove_footnotes(year)
        if year == "Year":
            continue
        rest = row[1].iloc[1]
        if pd.isna(rest):
            rest = ""
        rest = remove_footnotes(rest)
        date = date_merge(year, rest)
        date, remarks = date_clean(date)
        event = row[1].iloc[2]
        if pd.isna(event):
            continue
        event2 = row[1].iloc[3]
        if pd.isna(event2) is False:
            if event.endswith('.') is False:
                event = event+"."
            event = f"{event} {event2}"
        event = remove_footnotes(event)
        if pd.isna(event):
            continue
        if remarks != "":
            event = f"Date: {remarks}, {event}"
        print(f"| {date} | {event} |")

| Date | Post-classical history event |
| ---- | ---- |
| 400 | Date: c., Highland Maya fall to the lowland city of Teotihuacan. Begins the decline of Maya culture and language in some parts of the highlands. |
| 405 | St. Jerome finished the Vulgate. The Christian Gospel is translated into Latin. |
| 410-08-24 | Rome is sacked by Alaric, King of the Visigoths. Decisive event in the decline of the Western Roman Empire. |
| 431-06-22 | Council of Ephesus. Confirmed the original Nicene Creed, and condemned the teachings of Nestorius, Patriarch of Constantinople, that led to his exile and separation with the Church of the East. |
| 455-06-02 | Rome is sacked by Genseric, King of the Vandals.. Another decisive event in the Fall of Rome and held by some historians to mark the "end of the Roman Empire". |
| 476-09-04 | Odoacer deposes the Roman Emperor Romulus Augustulus. Considered by some historians to be the starting point of the Middle Ages. |
| 480-04-25 | Death of Julius Nepos, last Ro

In [ ]:
# https://en.wikipedia.org/wiki/Estimates_of_historical_world_population
# https://en.wikipedia.org/wiki/Prehistoric_demography
# https://en.wikipedia.org/wiki/Neolithic_decline

# https://en.wikipedia.org/wiki/List_of_epidemics_and_pandemics

# https://en.wikipedia.org/wiki/Holocene_climatic_optimum
# https://en.wikipedia.org/wiki/Roman_Warm_Period
# https://en.wikipedia.org/wiki/Late_Antique_Little_Ice_Age
# https://en.wikipedia.org/wiki/Medieval_Warm_Period
# https://en.wikipedia.org/wiki/Historical_climatology

# https://en.wikipedia.org/wiki/Timeline_of_international_trade

In [9]:
url = "https://en.wikipedia.org/wiki/Timeline_of_Buddhism"
tables = pd.read_html(url)

In [20]:
tables[29]

,Date,Event
0,2001,May – Two of the world's tallest ancient Buddh...
1,2002,"Khenmo Drolma, an American woman, became the f..."
2,2003,Ayya Sudhamma Bhikkhuni became the first Ameri...
3,2004,"Khenmo Drolma became the first westerner, male..."
4,2004,"April – In Sri Lanka, Buddhist monks acting as..."
5,2006,"March – Merle Kodo Boyd, born in Texas, became..."
6,2006,April – The Government of the People's Republi...
7,2006,"For the first time in American history, a Budd..."
8,2007,"Myokei Caine-Barrett, born and ordained in Jap..."
9,2008,After a 10-year process of advanced training c...


In [24]:
print("| Date | Buddhist history event |")
print("| ---- | ---- |")
for index in range(4, 30):
    for i, row in enumerate(tables[index].iterrows()):
        date = row[1].iloc[0]
        if pd.isna(date) or year == "nan":
            continue
        date, remarks = date_clean(date)
        event = row[1].iloc[1]
        if pd.isna(event):
            continue
        if remarks != "":
            event = f"Date: {remarks}, {event}"
        print(f"| {date} | {event} |")

| Date | Buddhist history event |
| ---- | ---- |
| 563 BC | Date: or  480 BC, The Birth of Siddhartha Gautama. The approximate date of Gautama Buddha's birth and death are uncertain. Most historians in the early 20th century dated his lifetime as circa 563 BCE to 483 BCE.[1][2] More recently his death is dated later, between 411 and 400 BCE, while at a symposium on this question held in 1988, the majority of those who presented definite opinions gave dates within 20 years either side of 400 BCE for the Buddha's death.[1][3] |
| 413 BC - 345 BC | Date: c., Shishunaga, a minister of the ruling Hiranyaka dynasty of Magadha, is placed on the throne and begins the Shishunaga dynasty, after the sitting king is deposed by the people. |
| 383 BC | Date: or  330 BC, The Second Buddhist council is convened by Kalasoka of the Shishunaga dynasty and held in Vaishali. The Sangha divides into the Sthaviravadins and the Mahasanghikas led by the monk Mahādeva, primarily over the question of addition 